In [3]:
import pandas as pd

df = pd.read_csv (r'SoldBuilding.csv')
df

,Unnamed: 0,building_name,sold_price,building_beds,building_baths,redfin_estimates,building_location,property_type,property_size,walking_score,transit_score,bike_score,building_description,building_website
0,0,9420 238th St,"570,000",4.0,2.0,"598,867","Floral Park,NY",Single Family Residential,"4,000",72.0,NaN,59.0,"""Won't Last! This Spacious 4-Bedroom, 2-Bath B...",https://www.redfin.com/NY/Floral-Park/9420-238...
1,1,95 Locustwood Blvd,"525,000",3.0,2.0,"559,541","Elmont,NY",Single Family Residential,"4,000",39.0,55.0,42.0,"""Magnificent Expanded Cape That's Much Larger ...",https://www.redfin.com/NY/Elmont/95-Locustwood...
2,2,852 Cypress Ave Unit 2F,"700,000",NaN,NaN,NaN,"Queens,NY",Condo/Co-op,NaN,99.0,98.0,81.0,NaN,https://www.redfin.com/NY/Queens/852-Cypress-A...
3,3,75-05 Utopia Pkwy,"775,000",3.0,3.0,"803,383","Flushing,NY",Single Family Residential,"1,440",90.0,63.0,64.0,"""Large home conveniently located in the heart ...",https://www.redfin.com/NY/Flushing/75-05-Utopi...
4,4,146-30 177th St,"900,000",6.0,4.0,"922,498","Jamaica,NY",Multi-Family (2-4 Unit),"4,000",64.0,62.0,50.0,"""Don't miss this meticulously maintained 3 ove...",https://www.redfin.com/NY/Queens/146-30-177th-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,331,415 Greenwich St Unit 3D,"2,999,000",NaN,NaN,NaN,"New York,NY",Condo/Co-op,NaN,97.0,100.0,90.0,NaN,https://www.redfin.com/NY/New-York/415-Greenwi...
332,332,10 W End Ave Unit 21B,"1,985,000",NaN,NaN,NaN,"New York,NY",Condo/Co-op,NaN,96.0,100.0,81.0,NaN,https://www.redfin.com/NY/New-York/10-W-End-Av...
333,333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.redfin.com/NY/New-York/80-Riversid...
334,334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.redfin.com/NY/New-York/35-Wooster-...


In [17]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [18]:
import requests # to get the website
import time     # to force our code to wait a little before re-trying to grab a webpage
import re       # to grab the exact element we need
from bs4 import BeautifulSoup # to grab the html elements we need
import pandas as pd # to create dataframe
from selenium import webdriver #use selenium to avoid website’s anti-scraping
from selenium.common.exceptions import NoSuchElementException
path = 'C:\\Users\\yugem\\webdrivers\\chromedriver.exe'
browser = webdriver.Chrome(path)
#In this data, we mainly focus on data in New York State
browser.get('https://www.redfin.com/state/New-York')
def check_exists_by_xpath(xpath):
    try:
        browser.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True
data = []
page_source = browser.page_source
soup = BeautifulSoup(page_source, 'lxml')
cities = 'NA'
avg_price = 'NA'
avg_ppersqr = 'NA'
avg_day = 'NA'
cities_link = 'NA'
if (check_exists_by_xpath('//a[@class="ui_button nav next primary "]')):
        browser.find_element_by_xpath('//a[@class="ui_button nav next primary "]').click()
        time.sleep(4)
table = soup.find('table')
tbody = table.find('tbody')
trs = tbody.find_all('tr')
for tr in trs:
    lnk = tr.find_all('a')
    lnk = lnk[0]
    link = lnk.get('href')
    cities_link = "https://www.redfin.com" + link
    cities = lnk.text.strip()
    cities = '"'+ cities + '"'
    td1 = tr.find_all('td',{'class':'c1'})
    td1 = td1[0]
    avg_price = td1.text.strip()
    avg_price = '"' + avg_price + '"'
    td2 = tr.find_all('td',{'class':'c2'})
    td2 = td2[0]
    avg_ppersqr = td2.text.strip()
    avg_ppersqr = '"' + avg_ppersqr + '"'
    td3 = tr.find_all('td',{'class':'c3'})
    td3 = td3[0]
    avg_day = td3.text.strip()
    avg_day = '"' + avg_day + '"'
    
    data.append([cities, avg_price, avg_ppersqr, avg_day, cities_link])  
#get each cities’ average price, average price/ sqr ft., average days on redfin, and link 
df = pd.DataFrame(data, columns = ['Cities', 'Avg.Price', 'Avg.price/sqr', 'Avg.days', 'link'])
#In this dataset, we only focus on NYC’s data
building_list = []
nyc = data[0][4]
webpages = 17
start = 1
for webpage in range(webpages):
    if (check_exists_by_xpath('//a[@class="ui_button nav next primary "]')):
        browser.find_element_by_xpath('//a[@class="ui_button nav next primary "]').click()
        time.sleep(4)
    pattern = '/page-'
    webpage = nyc + pattern + str(start)
    browser.get(webpage)
    page_source1 = browser.page_source
    soup1 = BeautifulSoup(page_source1, 'lxml')
    start += 1
    div = soup1.findAll('div', {'class':re.compile('HomeCardContainer')})
    for item in div:
        if (check_exists_by_xpath('//a[@class="ui_button nav next primary "]')):
            browser.find_element_by_xpath('//a[@class="ui_button nav next primary "]').click()
            time.sleep(4)
        a = item.find('a')
        if a == None:
            continue
        else:
            building_link = a.get('href')
            website_pattern = 'https://www.redfin.com'
            building_link = website_pattern + building_link
            browser.get(building_link)
            page_source2 = browser.page_source
            soup2 = BeautifulSoup(page_source2, 'lxml')
            span_address = soup2.find('span', {'class':re.compile('street-address')})
            building_name = span_address.text.strip()
            building_name = '"' + building_name + '"'
            div_price = soup2.find('div', {'class':re.compile('statsValue')})
            building_price = div_price.text.strip()
            building_price = '"' + building_price + '"'
            div_beds = soup2.find('div', {'data-rf-test-id':'abp-beds'})
            building_beds = div_beds.text.strip()
            if building_beds == '—Beds':
                building_beds = 'NA'
            else:
                building_beds = '"' + building_beds + '"'
            div_baths = soup2.find('div', {'data-rf-test-id':'abp-baths'})
            building_baths = div_baths.text.strip()
            if building_baths == '—Baths':
                building_baths = 'NA'
            else:
                building_baths = '"' + building_baths + '"'
            description_p = soup2.find('p', {'class':re.compile('text-base')})
            if description_p == None:
                building_description = 'NA'
            else:
                building_description = description_p.text.strip()
                building_description = '"' + building_description + '"'
            span_estimate = soup2.find('span', {'data-rf-test-id': 'avmLdpPrice'})
            if span_estimate == None:
                redfin_estimates = 'NA'
            else:
                subspan_estimate = span_estimate.find('span', {'class':re.compile('value')})
                redfin_estimates = subspan_estimate.text.strip()
                redfin_estimates = '"' + redfin_estimates + '"'
            span_location = soup2.find('span', {'class':re.compile('locality')})
            span_region = soup2.find('span', {'class':re.compile('region')})
            building_location = span_location.text.strip()
            building_region = span_region.text.strip()
            building_location = '"' + building_location + building_region + '"'
            div_type = soup2.findAll('div', {'class':re.compile('table-value')})
            if len(div_type) == 0:
                property_type = 'NA'
                property_size = 'NA'
                property_story = 'NA'
            else:
                property_type = div_type[7].text.strip()
                if property_type == '—':
                    property_type = 'NA'
                else:
                    property_type = '"' + property_type + '"'
                property_size = div_type[4].text.strip()
                if property_size == '—':
                    property_size = 'NA'
                else:
                    property_size = '"' + property_size + '"'
                property_story = div_type[5].text.strip()
                if property_story == '—':
                    property_size = 'NA'
                else:
                    property_story = '"' + property_story + '"'
            walking_div = soup2.find('div', {'class':re.compile('transport-icon-and-percentage walkscore')})
            if walking_div == None:
                walking_score = 'NA'
            else:
                walking_span = walking_div.find('span', {'class':re.compile('value')})
                walking_score = walking_span.text.strip()
                walking_score = '"' + walking_score + '"'
            transit_div = soup2.find('div', {'class':re.compile('transport-icon-and-percentage transitscore')})
            if transit_div == None:
                transit_score = 'NA'
            else:
                transit_span = transit_div.find('span', {'class':re.compile('value')})
                transit_score = transit_span.text.strip()
                transit_score = '"' + transit_score + '"'
            bike_div = soup2.find('div', {'class':re.compile('transport-icon-and-percentage bikescore')})
            if bike_div == None:
                bike_score = 'NA'
            else:
                bike_span = bike_div.find('span', {'class':re.compile('value')})
                bike_score = bike_span.text.strip()
                bike_score = '"' + bike_score + '"'
            
            building_list.append([
                building_name, building_price, building_beds, 
                building_baths, redfin_estimates, building_location, 
                property_type, property_size, property_story, 
                walking_score, transit_score, bike_score, building_description, building_link])
#create dataframe for building_list of active listing building’s information
df1 = pd.DataFrame(building_list, columns = ['building_name', 'building_price', 'building_beds', 
                'building_baths', 'redfin_estimates', 'building_location', 
                'property_type', 'property_size', 'property_story', 'walking_score', 'transit_score',
                                             'bike_score', 'building_description', 'building_website'])

df1.head(10)
sold_list = []
webpages = 17
filter_pattern = '/filter/include=sold-3mo'
start = 1
for webpage in range(webpages):
    if (check_exists_by_xpath('//a[@class="ui_button nav next primary "]')):
        browser.find_element_by_xpath('//a[@class="ui_button nav next primary "]').click()
        time.sleep(4)
    pattern = '/page-'
    webpage1 = nyc + filter_pattern + pattern + str(start)
    browser.get(webpage1)
    page_source3 = browser.page_source
    soup3 = BeautifulSoup(page_source3, 'lxml')
    start += 1
    divs = soup3.findAll('div', {'class':re.compile('HomeCardContainer')})
    for item in divs:
        if (check_exists_by_xpath('//a[@class="ui_button nav next primary "]')):
            browser.find_element_by_xpath('//a[@class="ui_button nav next primary "]').click()
            time.sleep(4) 
        a = item.find('a')
        if a == None:
            continue
        else:
            sold_building_link = a.get('href')
            website_pattern = 'https://www.redfin.com'
            sold_building_link = website_pattern + sold_building_link
            browser.get(sold_building_link)
            page_source4 = browser.page_source
            soup4 = BeautifulSoup(page_source4, 'lxml')
            span_address = soup4.find('span', {'class':re.compile('street-address')})
            if span_address == None:
                building_name = 'NA'
            else:
                building_name = span_address.text.strip()
                building_name = '"' + building_name + '"'
            div_price = soup4.find('div', {'data-rf-test-id':'abp-price'})
            if div_price == None:
                sold_price = 'NA'
            else:
                subdiv_price = div_price.find('div')
                sold_price = subdiv_price.text.strip()
                sold_price = '"' + sold_price + '"'
            redfin_div = soup4.find('div', {'data-rf-test-id':'avm-price'})
            if redfin_div == None:
                redfin_estimates = 'NA'
            else:
                subredfin_div = redfin_div.find('div')
                redfin_estimates = subredfin_div.text.strip()
                redfin_estimates = '"' + redfin_estimates + '"'
            listed_div = soup4.find('div', {'class':re.compile('price-col number')})
            if listed_div == None:
                listed_price = 'NA'
            else:
                listed_price = listed_div.text.strip()
                listed_price = '"' + listed_price + '"'
            div_beds = soup4.find('div', {'data-rf-test-id':'abp-beds'})
            if div_beds == None:
                building_beds = 'NA'
            else:
                building_beds = div_beds.text.strip()
            if building_beds == '—Beds':
                building_beds = 'NA'
            else:
                building_beds = '"' + building_beds + '"'
            div_baths = soup4.find('div', {'data-rf-test-id':'abp-baths'})
            if div_baths == None:
                building_baths = 'NA'
            else:
                building_baths = div_baths.text.strip()
            if building_baths == '—Baths':
                building_baths = 'NA'
            else:
                building_baths = '"' + building_baths + '"'
            description_p = soup4.find('p', {'class':re.compile('text-base')})
            if description_p == None:
                building_description = 'NA'
            else:
                building_description = description_p.text.strip()
                building_description = '"' + building_description + '"'
            span_location = soup4.find('span', {'class':re.compile('locality')})
            if span_location == None:
                building_location = 'NA'
            else:
                span_region = soup4.find('span', {'class':re.compile('region')})
                building_location = span_location.text.strip()
                building_region = span_region.text.strip()
                building_location = '"' + building_location + building_region + '"'
            div_type = soup4.findAll('div', {'class':re.compile('table-value')})
            if len(div_type) == 0:
                property_type = 'NA'
                property_size = 'NA'
                property_story = 'NA'
            else:
                property_type = div_type[5].text.strip()
                if property_size == '—':
                    property_size = 'NA'
                else:
                    property_type = '"' + property_type + '"'
                property_size = div_type[4].text.strip()
                if property_size == '—':
                    property_size = 'NA'
                else:
                    property_size = '"' + property_size + '"'
                property_story = div_type[5].text.strip()
                if property_story == '—':
                    property_story = 'NA'
                else:
                    property_story = '"' + property_story + '"'
            walking_div = soup4.find('div', {'class':re.compile('transport-icon-and-percentage walkscore')})
            if walking_div == None:
                walking_score = 'NA'
            else:
                walking_span = walking_div.find('span', {'class':re.compile('value')})
                walking_score = walking_span.text.strip()
                walking_score = '"' + walking_score + '"'
            transit_div = soup4.find('div', {'class':re.compile('transport-icon-and-percentage transitscore')})
            if transit_div == None:
                transit_score = 'NA'
            else:
                transit_span = transit_div.find('span', {'class':re.compile('value')})
                transit_score = transit_span.text.strip()
                transit_score = '"' + transit_score + '"'
            bike_div = soup4.find('div', {'class':re.compile('transport-icon-and-percentage bikescore')})
            if bike_div == None:
                bike_score = 'NA'
            else:
                bike_span = bike_div.find('span', {'class':re.compile('value')})
                bike_score = bike_span.text.strip()
                bike_score = '"' + bike_score + '"'
                 
            sold_list.append([
                building_name, sold_price, building_beds, 
                building_baths, redfin_estimates, listed_price, building_location, 
                property_type, property_size, property_story, 
                walking_score, transit_score, bike_score, building_description, sold_building_link])
#create dataframe for building_list of sold building’s information
df2 = pd.DataFrame(sold_list, columns = ['building_name', 'sold_price', 'building_beds', 'building_baths',
                                         'redfin_estimates', 'listed_price','building_location', 
                'property_type', 'property_size', 'property_story', 
                                         'walking_score', 'transit_score', 'bike_score',
                                         'building_description', 'building_website'])
df2.head(10)



WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
